In [1]:
# from models.sos import ScalarOnScalarModel
# from optimizers.nbdo import NBDO
# from bases.bspline import BSplineBasis
# from models.sof import ScalarOnFunctionModel
# import numpy as np

In [2]:
# # SoS
# model = ScalarOnScalarModel(Kx=3, criterion="A", order=2)
# nbdo = NBDO(model=model, latent_dim=10, seed=42)
# nbdo.compute_train_set(num_designs=1_000, runs=25)
# nbdo.fit(epochs=1_000, patience=100, batch_size=256)
# crit, design = nbdo.optimize(n_calls=30)
# np.round(crit,2)

In [3]:
# # SoF
# xB, bB = BSplineBasis(0,4), BSplineBasis(0,2)
# model = ScalarOnFunctionModel([(xB,bB)], criterion="D", intercept=True)
# nbdo = NBDO(model=model, latent_dim=2, seed=42, verbose=False)
# nbdo.compute_train_set(num_designs=1_000, runs=12)
# nbdo.fit(epochs=10, patience=5, batch_size=256)
# crit, design = nbdo.optimize(n_calls=5)
# crit

In [4]:
# from diagnostics import (
#     info_matrix, eigen_spectrum, condition_number,
#     leverage_diag, prediction_variance,
# )

# M = info_matrix(model, design)
# eigvals, _ = eigen_spectrum(M)
# kappa = condition_number(M)
# print(f"p = {M.shape[0]}, λ_min = {eigvals[-1]:.3e}, λ_max = {eigvals[0]:.3e}, κ2 = {kappa:.2e}")

# h = leverage_diag(model, design)
# print(f"leverage stats: min={h.min():.3f}, max={h.max():.3f}, mean={h.mean():.3f}")

In [5]:
# p = 10, λ_min = 3.458e+00, λ_max = 5.728e+01, κ2 = 1.66e+01
# leverage stats: min=0.179, max=0.748, mean=0.400

In [6]:
# # quick smoke test
# from bases.bspline import BSplineBasis
# from models.fof import FunctionOnFunctionModel
# from optimizers.nbdo import NBDO

# def B(K, deg):
#     return BSplineBasis(degree=deg, total_knots_num=max(2, K - deg + 1))

# bx, bb = B(5,2), B(6,2)
# model = FunctionOnFunctionModel([(bx, bb)], criterion="A", intercept=True)
# opt = NBDO(model=model, latent_dim=4, seed=0)
# opt.compute_train_set(num_designs=1_000, runs=12)
# opt.fit(epochs=1_000, patience=100, batch_size=256)       # should converge & print losses
# report, design = opt.optimize(n_calls=5, n_random_starts=3)
# print("ok ✅", report, design.shape)

In [7]:
# import numpy as np
# from bases.bspline import BSplineBasis
# from models.sof import ScalarOnFunctionModel
# from models.fof import FunctionOnFunctionModel

# def B(K, deg):
#     total_knots_num = max(2, K - deg + 1)
#     return BSplineBasis(degree=deg, total_knots_num=total_knots_num)

# # Two predictors, different dims
# bx1, bb1 = B(5,2), B(6,2)
# bx2, bb2 = B(4,3), B(3,3)
# pairs = [(bx1, bb1), (bx2, bb2)]

# runs = 14

# fof = FunctionOnFunctionModel(pairs, criterion="A", intercept=True)
# sof = ScalarOnFunctionModel(pairs, criterion="A", intercept=True)

# # Build Γ by concatenating per-predictor blocks
# rng = np.random.default_rng(0)
# Gamma = np.hstack([
#     rng.normal(size=(runs, fof.Kx_list[0])),
#     rng.normal(size=(runs, fof.Kx_list[1])),
# ])

# # Block-diag sanity: each slice maps correctly
# Phi = Gamma @ fof.J_np
# for xs, bs in zip(fof.x_slices, fof.b_slices):
#     Ji = fof.J_np[xs, :][:, bs]
#     assert np.allclose(Gamma[:, xs] @ Ji, Phi[:, bs], atol=1e-12)

# # Shapes and parity
# Zf, Zs = fof.model_matrix(Gamma), sof.model_matrix(Gamma)
# Mf, Ms = fof.information_matrix(Zf), sof.information_matrix(Zs)
# print("Z shapes:", Zf.shape, Zs.shape)
# print("M shapes:", Mf.shape, Ms.shape)

# vf, vs = fof.objective_num(Gamma), sof.objective_num(Gamma)
# print("A-opt FoF vs SoF:", vf, vs)
# assert np.allclose(vf, vs, rtol=1e-10, atol=1e-10)

# print("Multi-predictor behavior matches SoF ✅")

In [8]:
# --- Scalar-on-Function parity baseline ---

import numpy as np
import tensorflow as tf

from bases.bspline import BSplineBasis
from models.sof import ScalarOnFunctionModel

# reproducibility
rng = np.random.default_rng(12345)
tf.keras.utils.set_random_seed(12345)

def B(K: int, degree: int) -> BSplineBasis:
    """
    Your BSplineBasis signature: (degree, total_knots_num)
    Relation: K = degree + total_knots_num - 1  -> total_knots_num = K - degree + 1
    """
    total_knots_num = max(2, K - degree + 1)
    return BSplineBasis(degree=degree, total_knots_num=total_knots_num)

# --- common design setup (multi-predictor) ---
deg1, deg2 = 2, 2
Kx1, Kb1 = 5, 6    # predictor 1: design basis size, beta(s,·) s-basis size
Kx2, Kb2 = 4, 3    # predictor 2: design basis size, beta(s,·) s-basis size

bx1, bb1 = B(Kx1, deg1), B(Kb1, deg1)
bx2, bb2 = B(Kx2, deg2), B(Kb2, deg2)
pairs = [(bx1, bb1), (bx2, bb2)]

intercept = True
Kb_total = Kb1 + Kb2
p = Kb_total + (1 if intercept else 0)

runs = 14  # ensure runs > p for a well-conditioned M

# --- build model(s) ---
sof_A = ScalarOnFunctionModel(pairs, criterion="A", intercept=intercept, dtype=tf.float64)
sof_D = ScalarOnFunctionModel(pairs, criterion="D", intercept=intercept, dtype=tf.float64)

# --- design coefficients Γ: (runs, Kx_total) ---
Gamma = rng.normal(size=(runs, sof_A.Kx))

# --- shapes & quick info ---
print("=== Scalar-on-Function (baseline) ===")
print(f"Kx_list={sof_A.Kx_list}, sum={sof_A.Kx} | Kb_list={sof_A.Kb_list}, sum={sof_A.Kb}")
print(f"p={sof_A.p} (should equal {p}), runs={runs}")
print("J shape:", sof_A.J_np.shape)

Z = sof_A.model_matrix(Gamma)
M = sof_A.information_matrix(Z)
print("Z shape:", Z.shape, "| M shape:", M.shape)

# --- criteria values ---
A_val = sof_A.objective_num(Gamma)
D_val = sof_D.objective_num(Gamma)

print("\nResults to copy:")
print(f"A-opt (trace(M^{-1})) loss: {A_val:.12f}")
print(f"D-opt (-logdet M) loss:     {D_val:.12f}")


=== Scalar-on-Function (baseline) ===
Kx_list=[5, 4], sum=9 | Kb_list=[6, 3], sum=9
p=10 (should equal 10), runs=14
J shape: (9, 9)
Z shape: (14, 10) | M shape: (10, 10)

Results to copy:
A-opt (trace(M^-1)) loss: 1000606.906679542270
D-opt (-logdet M) loss:     31.342415285497


In [ ]:
# # --- Function-on-Function parity check (should match SoF numbers) ---

# import numpy as np
# import tensorflow as tf

# from bases.bspline import BSplineBasis
# from models.fof import FunctionOnFunctionModel

# # reproducibility (same seeds as in Cell 1)
# rng = np.random.default_rng(12345)
# tf.keras.utils.set_random_seed(12345)

# def B(K: int, degree: int) -> BSplineBasis:
#     total_knots_num = max(2, K - degree + 1)
#     return BSplineBasis(degree=degree, total_knots_num=total_knots_num)

# # --- same common design setup (multi-predictor) ---
# deg1, deg2 = 2, 2
# Kx1, Kb1 = 5, 6
# Kx2, Kb2 = 4, 3

# bx1, bb1 = B(Kx1, deg1), B(Kb1, deg1)
# bx2, bb2 = B(Kx2, deg2), B(Kb2, deg2)
# pairs = [(bx1, bb1), (bx2, bb2)]

# intercept = True
# Kb_total = Kb1 + Kb2
# p = Kb_total + (1 if intercept else 0)

# runs = 14  # same as Cell 1

# # --- build model(s) ---
# fof_A = FunctionOnFunctionModel(pairs, criterion="A", intercept=intercept, dtype=tf.float64, response_basis=None)
# fof_D = FunctionOnFunctionModel(pairs, criterion="D", intercept=intercept, dtype=tf.float64, response_basis=None)

# # --- design coefficients Γ: (runs, Kx_total) -- same RNG/state as Cell 1 ---
# Gamma = rng.normal(size=(runs, fof_A.Kx))

# # --- shapes & quick info ---
# print("=== Function-on-Function (to match SoF) ===")
# print(f"Kx_list={fof_A.Kx_list}, sum={fof_A.Kx} | Kb_list={fof_A.Kb_list}, sum={fof_A.Kb}")
# print(f"p={fof_A.p} (should equal {p}), runs={runs}")
# print("J shape:", fof_A.J_np.shape)

# Z = fof_A.model_matrix(Gamma)
# M = fof_A.information_matrix(Z)
# print("Z shape:", Z.shape, "| M shape:", M.shape)

# # --- criteria values (should match those you copied from Cell 1) ---
# A_val = fof_A.objective_num(Gamma)
# D_val = fof_D.objective_num(Gamma)

# print("\nCompare to your SoF notes:")
# print(f"A-opt (trace(M^{-1})) loss: {A_val:.12f}")
# print(f"D-opt (-logdet M) loss:     {D_val:.12f}")


=== Function-on-Function (to match SoF) ===
Kx_list=[5, 4], sum=9 | Kb_list=[6, 3], sum=9
p=10 (should equal 10), runs=14
J shape: (9, 9)
Z shape: (14, 10) | M shape: (10, 10)

Compare to your SoF notes:
A-opt (trace(M^-1)) loss: 1000606.906679542270
D-opt (-logdet M) loss:     31.342415285497
